最初に今週必要なPythonパッケージをインストールします。

```bash
$ git pull
$ pipenv install
```

In [ ]:
%load_ext expectmagic

# 前処理（続き）

## ゴミの除去
### 正規表現（続き）

### 実習: Wikipediaのダンプの内部リンクをプレーンテキストに変換しよう

復習ともう少し複雑な正規表現の処理を兼ねて。

In [ ]:
# 実習用ファイルのダウンロード
from urllib import request

with open('data/uk.txt', 'wb') as f, request.urlopen('https://gist.githubusercontent.com/pizzacat83/9a9609d393c251b8df8e5bde96cd38c6/raw/141e1a9344e1cc7a8515e0ab85e57a92733435b7/uk.txt') as req:
    f.write(req.read())


In [ ]:
### tutor-only

import os
assert os.path.exists('data/uk.txt')

#### 1. ファイル読み込み（復習）
`data/uk.txt`を読み込んで，`text`という変数にファイルの内容を代入してください。その後，`text`を表示してください。

- ファイルを開く関数`open('ファイルパス', 'モード (optional)')`
- 一気にファイル全体を読みこむメソッド`f.read()`

In [ ]:
### answer

f = open('data/uk.txt')
text = f.read()
print(text)

`[[ほにゃらら]]`という形で内部リンクが表示されています。これを`ほにゃらら`に変換していきます。雑にやるなら`[[`と`]]`を全部削除してしまうわけですが，今回は対応している括弧だけを除去したいという設定でいきます。

#### 内部リンクの検出

**タスク**: `text`に含まれる`[[...]]`に，括弧ごとマッチするような正規表現`r`を作成してください。その後，その正規表現にマッチする文字列を全て列挙してください。

[Python公式チュートリアル](https://docs.python.org/ja/3/howto/regex.html)

- まず`re`をインポート
- 正規表現は`re.compile('source')`で作る
- 「どんな文字でもいい，何文字でもいい」: `.*`
  - 最短マッチにするなら`.*?`
- パターンを全て検索するには`r.findall('検索対象文字列')`


- `[tsg]`: `t`, `s`, `g`のいずれか1文字にマッチ
- `.`: 改行以外の任意の1文字にマッチ
- `*`: 0回以上の繰り返し
- `+`: 1回以上の繰り返し
- `?`: `*` `+`の後につけると最左マッチになる（デフォルトは最長マッチ）
- `pattern1|pattern2`: `pattern1`または`pattern2`にマッチ
- `(pattern)`: グルーピング。演算子の優先度を変えるのにも使えるし，`()`内にマッチしたテキストを得ることもできる
- `\`: 特殊文字のエスケープ

In [ ]:
%%expect ['[[イングランド]]', '[[スコットランド]]', '[[ウェールズ]]', '[[北アイルランド]]', '[[ロンドン]]', '[[エディンバラ]]', '[[カーディフ]]', '[[ベルファスト]]']
### answer

import re

#r = re.compile('\\[\\[.+?\\]\\]')
r = re.compile('\[\[.+?\]\]')

r.findall(text)

**注意点**
- `[`と`]`は`[tsg]`で使う特殊文字なので，文字としての`[`を指定したいときは`/`でエスケープ
- `'\['`ではpythonが`\`をpythonの文字列のエスケープ文字`\`として認識してしまう
  + `\n`と表示される文字列を作ろうとして`'\n'`とすると改行になってしまうので，`'\\n'`と書くのと同じ
  + つまり`'\\['`と書けば良い
  + 正規表現ではこうしたダブルバックスラッシュを多用するので，代わりに`\`を文字の`\`として扱ってくれる文字列リテラル`r'...'`を使うことも

#### 括弧の除去
前回は`《ルビ》`を丸ごと除去する（空文字列で置換する）という処理をしましたが，今回は丸ごと削除ではなく，括弧の中のテキストは残す必要があります。つまり，`[[ほにゃらら]]`を`ほにゃらら`で置換できれば良さそうです。

これを可能にするのが**グルーピング**です。検出した部分のうち括弧の中身だけを残したいので，括弧の中身をグルーピングします。グルーピングをするには，パターンを`()`で括ります。括った上で，まずは`findall`してみましょう。

In [ ]:
#%%expect ['イングランド', 'スコットランド', 'ウェールズ', '北アイルランド', 'ロンドン', 'エディンバラ', 'カーディフ', 'ベルファスト']
### answer

r = re.compile('\[\[(.+?)\]\]')

r.findall(text)

`[[ほにゃらら]]`を`ほにゃらら`に置換するのにも前回同様`r.sub('置換後', '対象文字列')`を使います。ただし，置換後に指定したいものはかっこの中身であり，マッチごとに異なります。置換後の文字列には先ほど作ったグループを指定することができます。かっこの中身は1番目のグループなので，`\1`と指定することで括弧の中身に置換することができます。

In [ ]:
# 例

r.sub('\\1', 'abc[[def]]g[[h]]i')

In [ ]:
### tutor-only

assert _ == 'abcdefghi'

**タスク**: `text`の`[[ほにゃらら]]`を`ほにゃらら`に置換してください。

In [ ]:
%%expect 'イギリスは四つの非独立国であるイングランド、スコットランド、ウェールズ、北アイルランドより構成される。それぞれの国は首都を持ち、ロンドン（イングランド）、エディンバラ（スコットランド）、カーディフ（ウェールズ）、ベルファスト（北アイルランド）がそれである。中でもイングランドの首都であるロンドンは、イギリスの首都としての機能も置かれる。'
### answer

r.sub('\\1', text)


## 正規化
本質的に区別する必要がない表層的な違いを統一します。

- アルファベットを全て小文字に
- 半角・全角の統一

ex. [たほいやの正規化関数](https://github.com/tsg-ut/slackbot/blob/513fcced5c957f6484f9e306e9b5586691eaa044/tahoiya/lib.js#L117-L129)

In [ ]:
# アルファベットの大文字・小文字変換

text = 'pizzacat83 lives in Japan.'

print(text)
print(text.lower()) # 小文字
print(text.upper()) # 大文字
print(text.capitalize()) # 1文字目だけ大文字
print(text.title()) # 単語頭だけ大文字

In [ ]:
# 半角・全角の統一

import jaconv

text = 'ぬるぽｶﾞｯＮｕｌｌPointer１２３４５67890'

print(text)
print(jaconv.h2z(text)) # Hankaku2Zenkaku デフォルトではカタカナのみ
print(jaconv.h2z(text, ascii=True, digit=True)) # 英数字も変換対象に
print(jaconv.z2h(text, ascii=True, digit=True)) # 全角→半角
print(jaconv.hira2kata(text)) # ひらがな→カタカナ

## 形態素解析
形態素解析とは，文字列を単語※単位に分割し，各単語の文法的情報を推定する処理です。日本語は単語を区切らないため，形態素解析によって単語の境界を検出する処理はほぼ必須です。単語区切りのみを目的に形態素解析をすることさえあります。

ちなみに英語は日本語と異なり単語をスペースで区切りますが，同じ綴りで品詞が異なる単語が多いため，品詞を推定するために形態素解析をすることがあります。

※正確には単語単位ではない

### インストール

まずはおそらく一番メジャーな形態素解析器であるMeCabをインストールしましょう。

#### Windows

[公式サイト](https://taku910.github.io/mecab/#download)からインストーラを落として実行。

#### Mac

```bash
$ brew install mecab
```

### 遊んでみよう

```bash
$ mecab
すもももももももものうち
すもも  名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も      助詞,係助詞,*,*,*,*,も,モ,モ
もも    名詞,一般,*,*,*,*,もも,モモ,モモ
も      助詞,係助詞,*,*,*,*,も,モ,モ
もも    名詞,一般,*,*,*,*,もも,モモ,モモ
の      助詞,連体化,*,*,*,*,の,ノ,ノ
うち    名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
EOS
```
Ctrl+Cで終了。

前処理が単語のスペース区切りで十分な場合も多く，これが手軽にできるオプションが存在します。
```
$ mecab -Owakati
すもももももももものうち
すもも も もも も もも の うち 
```

### 色々な形態素解析器
日本語の形態素解析器はMeCabだけではありません。

- [MeCab](https://taku910.github.io/mecab/) 速い
- [JUMAN](http://nlp.ist.i.kyoto-u.ac.jp/index.php?JUMAN) 同じスコアの最適解を出力
  + [demo](http://lotus.kuee.kyoto-u.ac.jp/nl-resource/cgi-bin/juman.cgi)
- [JUMAN++](http://nlp.ist.i.kyoto-u.ac.jp/index.php?JUMAN++) RNN 文全体を考慮した解析
  + [demo](http://tulip.kuee.kyoto-u.ac.jp/demo/jumanpp_lattice)
- [KyTea](http://www.phontron.com/kytea/index-ja.html) 点推定
  + [demo?](https://twitter.com/KyTeaJa)
- [kuromoji](https://www.atilika.com/ja/kuromoji/)

前処理に使う形態素解析器を選定する際には，形態素解析器の性能と辞書の性能を混同しないようにしましょう。例えばJUMANのデモの単語のドメインや代表表記に心惹かれたかもしれませんが，それは辞書にその情報が書かれているからであり，JUMANの解析アルゴリズムの性能ではありません。JUMANの辞書をMeCabで使えばMeCabの速度とJUMAN辞書の便利な情報をいいとこ取りできます。

### 実習: Pythonで形態素解析

形態素解析を使った前処理をPython上でやっていきます。`MeCab`というモジュールを使います。

In [ ]:
import MeCab

In [ ]:
tagger = MeCab.Tagger()

In [ ]:
text = 'メロスは激怒した。必ず、かの邪智暴虐の王を除かなければならぬと決意した。'

In [ ]:
print(tagger.parse(text)) # ターミナル上で実行するのと同じ

各単語について繰り返し処理をする際は，`tagger.parseToNode`を使用します。返り値は連結リストの先頭ノードで，少し慣れが必要かもしれません。

#### 0. parseToNodeの用例

In [ ]:
node = tagger.parseToNode(text)

while node.next: # 次のノードがなくなる（終端）まで
    node = node.next # 次のノードを見る
    
    print(node.surface, # 表層形
          node.feature) # 形態素の詳細

#### 1. 分かち書き
上のコードを参考にして，`text`を形態素解析した結果の**表層形**のリスト`wakati`を作成してください。

In [ ]:
### answer

node = tagger.parseToNode(text)
wakati = []

while node.next:
    node = node.next
    
    wakati.append(node.surface)

In [ ]:
%%expect ['メロス', 'は', '激怒', 'し', 'た', '。', '必ず', '、', 'かの', '邪智', '暴虐', 'の', '王', 'を', '除か', 'なけれ', 'ば', 'なら', 'ぬ', 'と', '決意', 'し', 'た', '。', '']

wakati

#### 2.  原形に正規化
`text`を形態素解析した結果の**原形**のリスト`wakati2`を作成してください。

`node.feature`には形態素解析結果が記録されています。この場合はカンマ区切りの文字列になっているため，ここから原形を抽出する必要があります。正規表現でも抽出は可能ですが，指定した文字で文字列を区切るだけなら`'string'.split('区切り文字')`というシンプルなメソッドが存在します。

In [ ]:
# 例（というよりヒント）

'動詞,自立,*,*,五段・カ行イ音便,未然形,除く,ノゾカ,ノゾカ'.split(',')

In [ ]:
### answer

node = tagger.parseToNode(text)
wakati2 = []

while node.next:
    node = node.next
    
    wakati2.append(node.feature.split(',')[6])

In [ ]:
%%expect ['*', 'は', '激怒', 'する', 'た', '。', '必ず', '、', 'かの', '邪智', '暴虐', 'の', '王', 'を', '除く', 'ない', 'ば', 'なる', 'ぬ', 'と', '決意', 'する', 'た', '。', '*']

wakati2

欠損値`'*'`を処理できるとなお良い。

#### 3. 名詞の抽出
textを形態素解析し，名詞のみの表層形からなるリスト`nouns`を作成してください。

In [ ]:
### answer

node = tagger.parseToNode(text)
nouns = []

while node.next:
    node = node.next
    if node.feature.split(',')[0] == '名詞':
        nouns.append(node.surface)

In [ ]:
%%expect ['メロス', '激怒', '邪智', '暴虐', '王', '決意']

nouns

## 構文解析
文節を単位にした係り受け解析などを行います。前処理として行うことは少ないので，紹介にとどめます。

### 主な構文解析器
- [CaboCha](https://taku910.github.io/cabocha/)
  + MeCab系
  + 形態素解析済みの入力も処理できる
  + 係り受け解析・固有表現認識
  + `brew install cabocha`
- [KNP](http://nlp.ist.i.kyoto-u.ac.jp/index.php?KNP)
  + JUMAN系
  + 係り受け解析・固有表現認識・格・照応解析
  + [demo](http://lotus.kuee.kyoto-u.ac.jp/nl-resource/cgi-bin/knp.cgi)